## Acerca del conjunto de datos

El analisis de personalidad de clientes es un analisis detallado de los clientes ideales de una empresa. Ayuda a la empresa a conocer mejor a sus clientes y les permite modificar los productos de acuerdo con las necesidades, comportmaientos e inquietudes de los diferentes tipos de clientes.

Ayuda a la empresa a modificar su producto en funcion de su cliente objetivo de distintos tipos de segmentos de clientes. Ejemplo, en lugar de gastar dinero para comercializar un nuevo producto a todos los clientes, una empresa puede analizar que segmento de clientes tienen mas probabilidades de comprar el producto y luego comercializarlo solo en ese segmento en particular

### Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from scipy.stats import boxcox, zscore
from scipy.special import inv_boxcox

from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans

### Cargar Datos

In [2]:
df = pd.read_csv('./marketing_campaign.csv', sep='\t')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [13]:
df.shape

(2240, 29)

> El dataset contiene 2240 observaciones y 29 caracteristicas

### Entendiendo los datos

In [5]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

__Atributos de las personas__

- ID: Identificador unico del cliente.
- Year_Birth: Fecha de nacimento del cliente.
- Education: Nivel educativo.
- Marital_Status: Estado civil.
- Income: Ingresos familiares anuales del cliente.
- Kidhome: Numero de niños en el hogar del cliente.
- Teenhome: Numero de adolescentes en el hogar del cliente.
- Dt_Customer: Fecha de alta del cliente en la empresa.
- Recency: Numero de dias desde la ultima compra del cliente.
- Complain: 1 si el cliente se quejo en los ultimos dos años, 0 en caso contrario.

__Atributos del producto__

- MntWines: Cantidad gastada en vino en los ultimos dos años.
- MntFruits: Cantidad gastada en frutas en los utlimos dos años.
- MntMeatProduts: Cantidad gastada en carnes en los utlimos dos años.
- MntFishProduct: Cantidad gastada en pescado en los utlimos dos años.
- MntSweetProduct: Cantidad gastada en dulces en los ultimos dos años.
- MntGoldProds: Cantidad gastada en oro en los ultimos dos años. 

__Atributos de la promocion__

- NumDealsPurchases: Numero de compras realizadas con descuento.
- AcceptedCmp1: 1 si el cliente acepto la oferta en la primera campaña, 0 en caso contrario.
- AcceptedCmp2: 1 si el cliente acepto la oferta en la segunda campaña, 0 en caso contrario.
- AcceptedCmp3: 1 si el cliente acepto la oferta en la tercera campaña, 0 en caso contrario.
- AcceptedCmp2: 1 si el cliente acepto la oferta en la cuarta campaña, 0 en caso contrario.
- AcceptedCmp2: 1 si el cliente acepto la oferta en la quinta campaña, 0 en caso contrario.
- Response: 1 si el cliente acepto la oferta en la ultima campaña, 0 en caso contrario.

__Atributos del lugar__

- NumWebPurchases: Numero de compras realizadas a traves del sitio web de la empresa.
- NumCatalogPurchases: Numero de compras realizadas por el catalogo de la empresa.
- NumStorePurchases: Numero de compras realizadas directamente en la tienda.
- NumWebVisitsMonth: Numero de visitas al sitio web de la empresa en el ultimo mes.

__Objetivo__

Realizar una agrupacion para resumir y comprender los segmentos de clientes y de esta forma identificar grupos distintos dentro de la base de clientes.

### Analisis Estadistico descriptivo

In [6]:
# mostrar las variables junto a su tipo de dato
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

> Lo anterior muestra que la variable _Income_ presenta valores faltantes. Debemos tratarlos.

Realizamos una resumen estadistico de las variables numericas

In [10]:
df.describe(include='number').round(2).T

,count,mean,std,min,25%,50%,75%,max
ID,2240.0,5592.16,3246.66,0.0,2828.25,5458.5,8427.75,11191.0
Year_Birth,2240.0,1968.81,11.98,1893.0,1959.00,1970.0,1977.00,1996.0
Income,2216.0,52247.25,25173.08,1730.0,35303.00,51381.5,68522.00,666666.0
Kidhome,2240.0,0.44,0.54,0.0,0.00,0.0,1.00,2.0
Teenhome,2240.0,0.51,0.54,0.0,0.00,0.0,1.00,2.0
Recency,2240.0,49.11,28.96,0.0,24.00,49.0,74.00,99.0
MntWines,2240.0,303.94,336.60,0.0,23.75,173.5,504.25,1493.0
MntFruits,2240.0,26.30,39.77,0.0,1.00,8.0,33.00,199.0
MntMeatProducts,2240.0,166.95,225.72,0.0,16.00,67.0,232.00,1725.0
MntFishProducts,2240.0,37.53,54.63,0.0,3.00,12.0,50.00,259.0


> El promedio de ingresos por cliente es de 52247.25 con una desviacion estandar de $\pm$ 	25173.08. Este resultado puede ser distinto debido a los datos que faltan.

> El ingreso minimo es de 1730 y el maximo es de 666666. El rango donde se encuenta gran parte de los datos viene dado por el promedio y la suma de la desviacion estandar. Dicho rango es de (27074.17, 77420.33). Al comparar los valores minimo y maximo con el rango, vemos que tales valores son atipicos.

> El promedio de niños en el hogar es de 0.44 lo que indica que gran parte de los hogares no tienen niños.

> El promedio de adolescentes en el hogar es de 0.51 lo que indica que gran parte de los hogares tienen al menos 1.

> El mayor ingreso para la empresa viene de las carnes y de los vinos.

Realizamos un resumen estadistico de las variable de tipo object

In [12]:
df.describe(include='object').T

,count,unique,top,freq
Education,2240,5,Graduation,1127
Marital_Status,2240,8,Married,864
Dt_Customer,2240,663,31-08-2012,12


> La mayor parte de los clientes son graduados y casados.

### Tratamiento de valores faltantes

In [14]:
# variable con su respectiva cantidad de valores nulos
df.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

> Hay 24 observaciones con valores faltantes en la variable _Income_. Esto representa el 1% del total de observaciones por lo que eliminamos tales registros.

In [15]:
# eliminar y actualizar el dataframe
df.dropna(inplace=True)
# mostrar nuevamente variables junto a su cantidad de valores faltantes
df.isnull().sum()

ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
dtype: int64

### Omitir columnas innecesarias

De las 29 caracteristicas, hay 3 que no aportan valor al analisis que estamos realizando y son: _ID_, _Z_Revenue_ y _Z_CostContact_. Elimanos estas caracteristicas y actualizamos el dataframe

In [18]:
df.drop(columns=['ID', 'Z_CostContact', 'Z_Revenue'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 0 to 2239
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           2216 non-null   int64  
 1   Education            2216 non-null   object 
 2   Marital_Status       2216 non-null   object 
 3   Income               2216 non-null   float64
 4   Kidhome              2216 non-null   int64  
 5   Teenhome             2216 non-null   int64  
 6   Dt_Customer          2216 non-null   object 
 7   Recency              2216 non-null   int64  
 8   MntWines             2216 non-null   int64  
 9   MntFruits            2216 non-null   int64  
 10  MntMeatProducts      2216 non-null   int64  
 11  MntFishProducts      2216 non-null   int64  
 12  MntSweetProducts     2216 non-null   int64  
 13  MntGoldProds         2216 non-null   int64  
 14  NumDealsPurchases    2216 non-null   int64  
 15  NumWebPurchases      2216 non-null   i

### Eliminar observaciones duplicadas

In [19]:
print(f'Cantidad de observaciones duplicadas: {(df.duplicated().sum())}')

Cantidad de observaciones duplicadas: 182


In [24]:
df.drop_duplicates(inplace=True)
df.shape

(2034, 26)